# LSST Twilight Planner (Modular) — Notebook

This notebook demonstrates the use of the `twilight_planner_pkg` package to
generate twilight observing plans. The workflow is:

1. Set up imports and paths for the input SN catalog and output directory.
2. Build a fully specified `PlannerConfig` object with all optional parameters
   shown (defaults are used unless otherwise noted).
3. Run the planner and inspect the CSV outputs written to `OUTDIR`.


In [ ]:
# --- Imports and path setup ---
import sys
import os
sys.path.insert(0, os.path.abspath(".."))

# Ensure local package is importable

from twilight_planner_pkg.config import PlannerConfig
from twilight_planner_pkg.scheduler import plan_twilight_range_with_caps

# Paths: adjust if your data/output lives elsewhere
CSV_PATH = "../data/ATLAS_2021_to25_with_fakes_like_input.csv"
OUTDIR = "../twilight_outputs"
os.makedirs(OUTDIR, exist_ok=True)

print("CSV exists:", os.path.exists(CSV_PATH))
print("Output dir:", OUTDIR)

## Configure the planner

The next cell defines a complete `PlannerConfig`. Every parameter of the
dataclass is included so the notebook doubles as documentation. Edit any value
to explore different strategies or hardware assumptions.

### Default Host Surface Brightness
- Exposure capping includes an optional default host SB fallback when per-target host inputs are missing.
- Enabled by default: `cfg.use_default_host_sb = True`.
- Default per-filter values: `cfg.default_host_mu_arcsec2_by_filter = {'u': 22.0, 'g': 22.0, 'r': 22.0, 'i': 22.0, 'z': 22.0, 'y': 22.0}`.
- To disable, set `cfg.use_default_host_sb = False`. Customize by editing `cfg.default_host_mu_arcsec2_by_filter`.


In [ ]:
# ---- User-editable parameters ----

# Date range (UTC)
START_DATE = "2023-01-01"
END_DATE = "2023-01-05"
RUN_LABEL = None

# -- Site --
LAT_DEG = -30.2446
LON_DEG = -70.7494
HEIGHT_M = 2663

# -- Visibility --
MIN_ALT_DEG = 20.0
TWILIGHT_SUN_ALT_MIN_DEG = -15.0  # minimum altitude for twilight
TWILIGHT_SUN_ALT_MAX_DEG = -5.0  # maximum altitude for twilight


# -- Filters and hardware --
FILTERS = ["r", "i", "z","y"]  # no "g" or "u" for twilight
CAROUSEL_CAPACITY = 5
FILTER_CHANGE_S = 120.0
READOUT_S = 2.0
FILTER_CHANGE_TIME_S = None  # legacy alias
READOUT_TIME_S = None  # legacy alias
INTER_EXPOSURE_MIN_S = 15.0
EXPOSURE_BY_FILTER = {"g": 5.0, "r": 5.0, "i": 5.0, "z": 5.0, "y": 5.0}

MAX_SWAPS_PER_WINDOW = 3

# NEW — cadence knobs
CADENCE_ENABLE = True
CADENCE_PER_FILTER = True
CADENCE_DAYS_TARGET = 3.0
CADENCE_JITTER_DAYS = 0.25
CADENCE_DAYS_TOLERANCE = 0.5
CADENCE_BONUS_SIGMA_DAYS = 0.5
CADENCE_BONUS_WEIGHT = 0.25

# Allow colors in one visit (if you want them)
MAX_FILTERS_PER_VISIT = 3  # was 1
ALLOW_FILTER_CHANGES_IN_TWILIGHT = True  # was False
START_FILTER = FILTERS[0]
SUN_ALT_POLICY = [
    (-18.0, -15.0, ["g", "r", "i", "z", "y"]),
    (-15.0, -12.0, ["r", "i", "z", "y"]),
    (-12.0, 0.0, ["i", "r", "z", "y"]),
]

# -- Slew model --
SLEW_SMALL_DEG = 3.5
SLEW_SMALL_TIME_S = 4.0
SLEW_RATE_DEG_PER_S = 5.25
SLEW_SETTLE_S = 1.0

# -- Moon --
MIN_MOON_SEP_BY_FILTER = {
    "u": 80.0,
    "g": 50.0,
    "r": 30.0,
    "i": 25.0,
    "z": 25.0,
    "y": 20.0,
}
REQUIRE_SINGLE_TIME_FOR_ALL = True

# -- Time caps --
PER_SN_CAP_S = 120.0
MORNING_CAP_S = 'auto'
EVENING_CAP_S = 'auto'
TWILIGHT_STEP_MIN = 2
MAX_SN_PER_NIGHT = 99999

# -- Priority strategy --
PRIORITY_STRATEGY = "hybrid"  # or "lc"
HYBRID_DETECTIONS = 2
HYBRID_EXPOSURE = 300.0
LC_DETECTIONS = 90
LC_EXPOSURE = 99999.0

# -- Photometry / sky --
PIXEL_SCALE_ARCSEC = 0.2
ZPT1S = None
K_M = None
FWHM_EFF = None
READ_NOISE_E = 6.0
GAIN_E_PER_ADU = 1.6
ZPT_ERR_MAG = 0.01
DARK_SKY_MAG = None
TWILIGHT_DELTA_MAG = 2.5  # fallback if rubin_sim.skybrightness unavailable

# -- SIMLIB output --
SIMLIB_OUT = f"twilight_{START_DATE}_to_{END_DATE}.simlib"
SIMLIB_SURVEY = "LSST"
SIMLIB_FILTERS = "grizY"
SIMLIB_PIXSIZE = 0.2
SIMLIB_NPE_PIXEL_SATURATE = 8e4
SIMLIB_PHOTFLAG_SATURATE = 4096
SIMLIB_PSF_UNIT = "arcsec"

# -- Miscellaneous --
TYPICAL_DAYS_BY_TYPE = {
    "Ia": 70,
    "II-P": 90,
    "II-L": 60,
    "IIn": 110,
    "IIb": 60,
    "Ib": 50,
    "Ic": 50,
}
DEFAULT_TYPICAL_DAYS = 60
RA_COL = None
DEC_COL = None
DISC_COL = None
NAME_COL = None
TYPE_COL = None

# Assemble the configuration object with all parameters exposed
# -- Low-z Ia policy (defaults for this notebook) --
LOW_Z_IA_MARKERS = ["ia", "1", "101"]
LOW_Z_IA_Z_THRESHOLD = 0.05
LOW_Z_IA_PRIORITY_MULTIPLIER = 10.0
LOW_Z_IA_CADENCE_DAYS_TARGET = 2.0
LOW_Z_IA_REPEATS_PER_WINDOW = 3

backfill_relax_cadence = True  # if True, relax cadence for backfilled visits if free time

cfg = PlannerConfig(
    lat_deg=LAT_DEG,
    lon_deg=LON_DEG,
    height_m=HEIGHT_M,
    min_alt_deg=MIN_ALT_DEG,
    twilight_sun_alt_min_deg=TWILIGHT_SUN_ALT_MIN_DEG,
    twilight_sun_alt_max_deg=TWILIGHT_SUN_ALT_MAX_DEG,
    filters=FILTERS,
    carousel_capacity=CAROUSEL_CAPACITY,
    filter_change_s=FILTER_CHANGE_S,
    readout_s=READOUT_S,
    filter_change_time_s=FILTER_CHANGE_TIME_S,
    readout_time_s=READOUT_TIME_S,
    inter_exposure_min_s=INTER_EXPOSURE_MIN_S,
    exposure_by_filter=EXPOSURE_BY_FILTER,
    max_filters_per_visit=MAX_FILTERS_PER_VISIT,
    max_swaps_per_window=MAX_SWAPS_PER_WINDOW,
    start_filter=START_FILTER,
    sun_alt_policy=SUN_ALT_POLICY,
    slew_small_deg=SLEW_SMALL_DEG,
    slew_small_time_s=SLEW_SMALL_TIME_S,
    slew_rate_deg_per_s=SLEW_RATE_DEG_PER_S,
    slew_settle_s=SLEW_SETTLE_S,
    min_moon_sep_by_filter=MIN_MOON_SEP_BY_FILTER,
    require_single_time_for_all_filters=REQUIRE_SINGLE_TIME_FOR_ALL,
    per_sn_cap_s=PER_SN_CAP_S,
    morning_cap_s=MORNING_CAP_S,
    evening_cap_s=EVENING_CAP_S,
    twilight_step_min=TWILIGHT_STEP_MIN,
    max_sn_per_night=MAX_SN_PER_NIGHT,
    cadence_enable=CADENCE_ENABLE,
    cadence_per_filter=CADENCE_PER_FILTER,
    cadence_days_target=CADENCE_DAYS_TARGET,
    cadence_jitter_days=CADENCE_JITTER_DAYS,
    cadence_days_tolerance=CADENCE_DAYS_TOLERANCE,
    cadence_bonus_sigma_days=CADENCE_BONUS_SIGMA_DAYS,
    cadence_bonus_weight=CADENCE_BONUS_WEIGHT,
    hybrid_detections=HYBRID_DETECTIONS,
    hybrid_exposure_s=HYBRID_EXPOSURE,
    lc_detections=LC_DETECTIONS,
    lc_exposure_s=LC_EXPOSURE,
    priority_strategy=PRIORITY_STRATEGY,
    pixel_scale_arcsec=PIXEL_SCALE_ARCSEC,
    zpt1s=ZPT1S,
    k_m=K_M,
    fwhm_eff=FWHM_EFF,
    read_noise_e=READ_NOISE_E,
    gain_e_per_adu=GAIN_E_PER_ADU,
    zpt_err_mag=ZPT_ERR_MAG,
    dark_sky_mag=DARK_SKY_MAG,
    twilight_delta_mag=TWILIGHT_DELTA_MAG,
    simlib_out=SIMLIB_OUT,
    simlib_survey=SIMLIB_SURVEY,
    simlib_filters=SIMLIB_FILTERS,
    simlib_pixsize=SIMLIB_PIXSIZE,
    simlib_npe_pixel_saturate=SIMLIB_NPE_PIXEL_SATURATE,
    simlib_photflag_saturate=SIMLIB_PHOTFLAG_SATURATE,
    simlib_psf_unit=SIMLIB_PSF_UNIT,
    low_z_ia_repeats_per_window=LOW_Z_IA_REPEATS_PER_WINDOW,
    low_z_ia_cadence_days_target=LOW_Z_IA_CADENCE_DAYS_TARGET,
    low_z_ia_priority_multiplier=LOW_Z_IA_PRIORITY_MULTIPLIER,
    low_z_ia_z_threshold=LOW_Z_IA_Z_THRESHOLD,
    low_z_ia_markers=LOW_Z_IA_MARKERS,
    typical_days_by_type=TYPICAL_DAYS_BY_TYPE,
    default_typical_days=DEFAULT_TYPICAL_DAYS,
    ra_col=RA_COL,
    dec_col=DEC_COL,
    disc_col=DISC_COL,
    name_col=NAME_COL,
    type_col=TYPE_COL,
    allow_filter_changes_in_twilight=ALLOW_FILTER_CHANGES_IN_TWILIGHT,
    only_ia= True,
    backfill_relax_cadence = backfill_relax_cadence,
    use_discovery_fallback=True,
    discovery_policy="atlas_priors",
    discovery_assumed_gr=0.0,
    discovery_margin_mag=0.2,
)

cfg

In [ ]:
# Diagnose catalog magnitude mapping for discovery fallback
from twilight_planner_pkg.examine import diagnose_mag_mapping
band_to_col, maglike_cols, preview = diagnose_mag_mapping(CSV_PATH, cfg)
print('Band→column mapping:', band_to_col)
print('Columns containing "mag":', maglike_cols[:10])
display(preview)



## Run the planner

`plan_twilight_range_with_caps` is invoked below.  It reads the SN catalog from
`CSV_PATH`, writes output CSVs to `OUTDIR`, and prints progress information.
Two files are produced for each run:

- `lsst_twilight_plan_<run_label>_*.csv` — per-SN schedule
- `lsst_twilight_summary_<run_label>_*.csv` — per-night summary


In [ ]:
# Execute the planner and return data frames
perSN_df, nights_df = plan_twilight_range_with_caps(
    csv_path=CSV_PATH,
    outdir=OUTDIR,
    start_date=START_DATE,
    end_date=END_DATE,
    cfg=cfg,
    run_label=RUN_LABEL,
    verbose=True,
)

print("Per-SN rows:", len(perSN_df), "  Nights summary rows:", len(nights_df))
perSN_df.head(10)

In [ ]:
perSN_df

In [ ]:
# Show the nightly summary
nights_df


## Inspect outputs on disk


In [ ]:
import glob
import os

files = sorted(glob.glob(os.path.join(OUTDIR, "lsst_twilight_*.csv")))
files

In [ ]:
# Diagnose per-visit saturation contributions
from twilight_planner_pkg.examine import build_saturation_df
saturation_df = build_saturation_df(perSN_df, CSV_PATH, cfg)
# Save and show a compact view
sat_path = os.path.join(OUTDIR, f'saturation_{START_DATE}_to_{END_DATE}.csv')
saturation_df.to_csv(sat_path, index=False)
cols = ['SN','filter','t_exp_s','t_exp_s_base','src_mag','sky_mag_arcsec2','host_mu_arcsec2',
        'e_src','e_sky','e_host','total_e','sat_limit','sat_guard_applied','warn_nonlinear',
        'over_sat','over_sat_base']
display(saturation_df[cols].head(20))
print('Wrote:', sat_path)

